In [1]:
import numpy as np
import tensorflow as tf
import h5py
import matplotlib.pyplot as plt
import matplotlib as mpl
import skimage
from PIL import Image
from skimage.measure import compare_ssim,compare_psnr
np.random.seed(73)

In [2]:
def compression(Data_in, code_in, stride_size, Q_scale, Q_type):
    compW = tf.constant(code_in)

    comp = tf.nn.conv2d(Data_in, compW, strides=[1, stride_size, stride_size, 1], padding='VALID')
    print(comp)

    compm = comp*Q_scale
    print(compm)

    compint = tf.cast(compm,Q_type)
    print(compint)
    
    return compint

In [3]:
block_size = 16
stride_size = 16
ncomp=4

Q_scale=1./2.

nrfs=10
Nx=2048
Ny=3840
NC = 1

#The 4 [16, 16] "masks" are stored in code[:,:,0,i] i=0,1,2,3
f_code=h5py.File('./code_16164_1x64_mix_4bit.h5','r')
code=f_code['data'][:]
code=code.astype(np.float32)
print("code shape: " + str(code.shape))

code shape: (16, 16, 1, 4)


In [4]:
with tf.name_scope('DATA'):
    Data_in = tf.placeholder(tf.float32,shape=[None,Nx,Ny,NC])
    print(Data_in)

Tensor("DATA/Placeholder:0", shape=(?, 2048, 3840, 1), dtype=float32)


In [5]:
comp_o=compression(Data_in, code, stride_size, Q_scale, tf.int8) #Note the output data type is signed 8-bit integer 
print(comp_o)

Tensor("Conv2D:0", shape=(?, 128, 240, 4), dtype=float32)
Tensor("mul:0", shape=(?, 128, 240, 4), dtype=float32)
Tensor("Cast:0", shape=(?, 128, 240, 4), dtype=int8)
Tensor("Cast:0", shape=(?, 128, 240, 4), dtype=int8)


In [6]:
def read_raw_data(fname):
    xs=0; ys=0
    xe=xs+2048; ye=ys+3840 #We choose only this range for test, you can change after you got familiar with the process
    pop=np.fromfile(fname,dtype=np.dtype('i2')) #Read .raw data
    pop=pop[0:3864*2174]
    
    pop=np.reshape(pop,(2174,3864))
    #pop=np.rot90(pop,k=2)  #For 1, 2, comment out if running 3, for exactly reproducing paper result
    
    pop=pop[xs:xe,ys:ye]
    pop=np.rot90(pop,k=2)  #For 3, comment out if running 1, 2, for exactly reproducing paper result
    
    #Divide by (2^14-1) so data_f has values between 0 and 1. You can do this overall division after Conv2D as well
    pop=pop/(np.power(2,14)-1.) 
    data_f=np.zeros([1,2048,3840,1])
    data_f[0,:,:,0]=pop
    
    return data_f

In [7]:
fileName='../Example_raw_bayer_data/3.raw'
data_in = read_raw_data(fileName)

In [8]:
print("data_in shape= " + str(data_in.shape) + " , max= " + str(np.max(data_in)) 
      + " , min= " + str(np.min(data_in)) + " , mean= " + str(np.mean(data_in)))

data_in shape= (1, 2048, 3840, 1) , max= 0.9999389611182323 , min= 0.06543368125495941 , mean= 0.19129893962974437


In [9]:
with tf.Session() as sess:
    comp_out=sess.run(comp_o,feed_dict={Data_in:data_in})

In [10]:
#comp_out is the output array of compression, in format signed 8-bit integer
print("comp_out shape= " + str(comp_out.shape) + " , type= " + str(comp_out.dtype) + " , max= " + str(np.max(comp_out)) 
      + " , min= " + str(np.min(comp_out)) + " , mean= " + str(np.mean(comp_out)))

comp_out shape= (1, 128, 240, 4) , type= int8 , max= 81 , min= -5 , mean= 14.727685546875


In [11]:
#Purely for storing in PNG files, add 128 to comp_out and put the values in unsigned 8-bit integers.
#When doing decompression, this 128 will be subtracted.
#If you choose another format to store the data and not adding 128 here, you need to do corresponding changes for decompression.
comp_u8 = comp_out + 128
comp_u8 = comp_u8.astype(np.uint8)
print("comp_u8 shape= " + str(comp_u8.shape) + " , type= " + str(comp_u8.dtype) + " , max= " + str(np.max(comp_u8)) 
      + " , min= " + str(np.min(comp_u8)) + " , mean= " + str(np.mean(comp_u8)))

comp_u8 shape= (1, 128, 240, 4) , type= uint8 , max= 209 , min= 123 , mean= 142.727685546875


In [12]:
#Just a check
np.mean(np.square(comp_u8-128-comp_out))

0.0

In [13]:
def trans_PNG(data,iname): #Save data as PNG file
    im = Image.fromarray(data)
    im.save(iname,compress_level=0)

In [14]:
i = 1
for j in range(4): #Store the convolutioin results using the "4 masks" separately in 4 PNG files
    imname = "./PNG_out/F" + str(i) + "_" + str(j) + ".png"
    print(imname)
    trans_PNG(comp_u8[0,:,:,j],imname)

./PNG_out/F1_0.png
./PNG_out/F1_1.png
./PNG_out/F1_2.png
./PNG_out/F1_3.png


In [15]:
#Just a check
i = 1
for j in range(4):
    imname = "./PNG_out/F" + str(i) + "_" + str(j) + ".png"
    print(imname)
    tim=Image.open(imname)
    tima=np.array(tim)
    mse=np.mean(np.square(tima-comp_u8[0,:,:,j]))
    print("  mse= " + str(mse) + " , png shape: " + str(tima.shape))

./PNG_out/F1_0.png
  mse= 0.0 , png shape: (128, 240)
./PNG_out/F1_1.png
  mse= 0.0 , png shape: (128, 240)
./PNG_out/F1_2.png
  mse= 0.0 , png shape: (128, 240)
./PNG_out/F1_3.png
  mse= 0.0 , png shape: (128, 240)


In [16]:
def write_hdf5(x,filename):
    with h5py.File(filename, 'w') as h:
        h.create_dataset('data', data=x, shape=x.shape)

In [17]:
write_hdf5(data_in,'image_orig.h5')